# Import and loading data


In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [6]:
!wget https://github.com/google-research-datasets/gap-coreference/raw/master/gap-development.tsv -q
!wget https://github.com/google-research-datasets/gap-coreference/raw/master/gap-test.tsv -q
!wget https://github.com/google-research-datasets/gap-coreference/raw/master/gap-validation.tsv -q

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

cuda:0


In [7]:
!pip install pytorch-pretrained-bert
!pip install https://github.com/ceshine/pytorch_helper_bot/archive/0.0.4.zip

     |████████████████████████████████| 133kB 12.2MB/s 
     |████████████████████████████████| 133kB 23.0MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 7.6MB 20.2MB/s 
ERROR: botocore 1.20.93 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     | 30kB 38.1MB/s
  Created wheel for PyTorchHelperBot: filename=PyTorchHelperBot-0.0.4-cp37-none-any.whl size=7180 sha256=fda5e5da4365fe9f50d2e4222f4eb17888a007e6666f94728a5ff7749212c5b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-8vfvsvh6/wheels/1f/01/01/da39a14e8e30666f3eec7106664e59059789c330a11b5fa357
Successfully built PyTorchHelperBot


In [8]:
import os

# This variable is used by helperbot to make the training deterministic
os.environ["SEED"] = "420"

import logging
from pathlib import Path

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel

from helperbot import BaseBot, TriangularLR, WeightDecayOptimizerWrapper

# Preparation functions

In [9]:
def insert_tag(row):
    """Insert custom tags to help us find the position of A, B, and the pronoun after tokenization."""
    to_be_inserted = sorted([
        (row["A-offset"], " [A] "),
        (row["B-offset"], " [B] "),
        (row["Pronoun-offset"], " [P] ")
    ], key=lambda x: x[0], reverse=True)
    text = row["Text"]
    for offset, tag in to_be_inserted:
        text = text[:offset] + tag + text[offset:]
    return text

def tokenize(text, tokenizer):
    """Returns a list of tokens and the positions of A, B, and the pronoun."""
    entries = {}
    final_tokens = []
    for token in tokenizer.tokenize(text):
        if token in ("[A]", "[B]", "[P]"):
            entries[token] = len(final_tokens)
            continue
        final_tokens.append(token)
    return final_tokens, (entries["[A]"], entries["[B]"], entries["[P]"])

class GAPDataset(Dataset):
    """Custom GAP Dataset class"""
    def __init__(self, df, tokenizer, labeled=True):
        self.labeled = labeled
        if labeled:
            tmp = df[["A-coref", "B-coref"]].copy()
            tmp["Neither"] = ~(df["A-coref"] | df["B-coref"])
            self.y = tmp.values.astype("bool")

        # Extracts the tokens and offsets(positions of A, B, and P)
        self.offsets, self.tokens = [], []
        for _, row in df.iterrows():
            text = insert_tag(row)
            tokens, offsets = tokenize(text, tokenizer)
            self.offsets.append(offsets)
            self.tokens.append(tokenizer.convert_tokens_to_ids(
                ["[CLS]"] + tokens + ["[SEP]"]))
        
    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        if self.labeled:
            return self.tokens[idx], self.offsets[idx], self.y[idx]
        return self.tokens[idx], self.offsets[idx]
    
def collate_examples(batch, truncate_len=500):
    """Batch preparation.
    
    1. Pad the sequences
    2. Transform the target.
    """
    transposed = list(zip(*batch))
    max_len = min(
        max((len(x) for x in transposed[0])),
        truncate_len
    )
    tokens = np.zeros((len(batch), max_len), dtype=np.int64)
    for i, row in enumerate(transposed[0]):
        row = np.array(row[:truncate_len])
        tokens[i, :len(row)] = row
    token_tensor = torch.from_numpy(tokens)
    # Offsets
    offsets = torch.stack([
        torch.LongTensor(x) for x in transposed[1]
    ], dim=0) + 1 # Account for the [CLS] token
    # Labels
    if len(transposed) == 2:
        return token_tensor, offsets, None
    one_hot_labels = torch.stack([
        torch.from_numpy(x.astype("uint8")) for x in transposed[2]
    ], dim=0)
    _, labels = one_hot_labels.max(dim=1)
    return token_tensor, offsets, labels

def collate_examples_no_labels(batch, truncate_len=500):
    """Batch preparation.
    
    1. Pad the sequences
    2. Transform the target.
    """
    transposed = list(zip(*batch))
    print("transposed size is", transposed.size())
    max_len = min(
        max((len(x) for x in transposed[0])),
        truncate_len
    )
    tokens = np.zeros((len(batch), max_len), dtype=np.int64)
    for i, row in enumerate(transposed[0]):
        row = np.array(row[:truncate_len])
        tokens[i, :len(row)] = row
    token_tensor = torch.from_numpy(tokens)
    # Offsets
    offsets = torch.stack([
        torch.LongTensor(x) for x in transposed[1]
    ], dim=0) + 1 # Account for the [CLS] token
    # Labels
    if len(transposed) == 2:
        return token_tensor, offsets, None
    one_hot_labels = torch.stack([
        torch.from_numpy(x.astype("uint8")) for x in transposed[2]
    ], dim=0)
    _, labels = one_hot_labels.max(dim=1)
    return token_tensor, offsets, labels


# Model

In [10]:
# Classical MLP model
class Head(nn.Module):
    """The MLP submodule"""
    def __init__(self, bert_hidden_size: int):
        super().__init__()
        self.head_hidden_size = 1024  # MLP hidden size
        self.bert_hidden_size = bert_hidden_size   # Bert hidden size
        self.fc = nn.Sequential(
            nn.BatchNorm1d(bert_hidden_size * 3),  # Batch Normalization * 3 is because the bert_output of A,B is expanded into 1 dimension
            nn.Dropout(0.5),                       # Random deactivation
            nn.Linear(bert_hidden_size * 3, self.head_hidden_size), # Linear layer
            nn.ReLU(),                                              # Activation function
            nn.BatchNorm1d(self.head_hidden_size),
            nn.Dropout(0.5),
            nn.Linear(self.head_hidden_size, self.head_hidden_size),
            nn.ReLU(),
            nn.BatchNorm1d(self.head_hidden_size),
            nn.Dropout(0.5),
            nn.Linear(self.head_hidden_size, self.head_hidden_size),
            nn.ReLU(),
            nn.BatchNorm1d(self.head_hidden_size),
            nn.Dropout(0.5),
            nn.Linear(self.head_hidden_size, 3)
        )
        
        # Parametrs inialization
        for i, module in enumerate(self.fc):
            if isinstance(module, (nn.BatchNorm1d, nn.BatchNorm2d)):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)
                print("Initing batchnorm")
            elif isinstance(module, nn.Linear):
                if getattr(module, "weight_v", None) is not None:
                    nn.init.uniform_(module.weight_g, 0, 1)
                    nn.init.kaiming_normal_(module.weight_v)
                    print("Initing linear with weight normalization")
                    assert model[i].weight_g is not None
                else:
                    nn.init.kaiming_normal_(module.weight)
                    print("Initing linear")
                nn.init.constant_(module.bias, 0)
    
    # Forward propagation
    def forward(self, bert_outputs, offsets):
        # bert_outputs:[batch_size, seq_length, hidden_szie]
        assert bert_outputs.size(2) == self.bert_hidden_size   
        
        # Taking out the embeddings at the offsets of A and B
        # unsqueeze(2):Expand 2-dimensional offsets to 3-dimensional
        # Extend a dimension of size 1. Such as (2,2,1) expands to (2,2,3)
        # input.gather(dim,index), index the specified dimension。For example, for a 4*3 tensor,indexing dim=1,then the value of index is 0~2.
        extracted_outputs = bert_outputs.gather(
            1, offsets.unsqueeze(2).expand(-1, -1, bert_outputs.size(2)) 
        ).view(bert_outputs.size(0), -1)      
        return self.fc(extracted_outputs)

# Current model
class GAPModel(nn.Module):
    """The main model."""
    def __init__(self, bert_model: str, device: torch.device):
        super().__init__()
        self.device = device  # Setting GPU device
        if bert_model in ("bert-base-uncased", "bert-base-cased"):
            self.bert_hidden_size = 768
        elif bert_model in ("bert-large-uncased", "bert-large-cased"):
            self.bert_hidden_size = 1024
        else:
            raise ValueError("Unsupported BERT model.")
        self.bert = BertModel.from_pretrained(bert_model).to(device)
        self.head = Head(self.bert_hidden_size).to(device)
    
    def forward(self, token_tensor, offsets):
        token_tensor = token_tensor.to(self.device)
        bert_outputs, _ =  self.bert(
            token_tensor, attention_mask=(token_tensor > 0).long(), 
            token_type_ids=None, output_all_encoded_layers=False)
        head_outputs = self.head(bert_outputs, offsets.to(self.device))
        return head_outputs             

In [12]:
offsets = torch.tensor([[0,1,2],[1,2,3]])  # batch_size=2, len(A,B,P)= 3
print(offsets.shape)
offsets = offsets.unsqueeze(2)
print(offsets)
print(offsets.shape)

torch.Size([2, 3])
tensor([[[0],
         [1],
         [2]],

        [[1],
         [2],
         [3]]])
torch.Size([2, 3, 1])


In [13]:
offsets=offsets.expand(-1,-1,5)   # 假设bert_hidden_size=5
print(offsets)

tensor([[[0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2]],

        [[1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3]]])


In [14]:
bert_outputs = torch.tensor([[[ 1,  2,  3,  4,  5],
                  [ 6,  7,  8,  9, 10],
                  [11, 12, 13, 14, 15],
                  [16, 17, 18, 19, 20]],
                 [[21, 22, 23, 24, 25],
                  [26, 27, 28, 29, 30],
                  [31, 32, 33, 34, 35],
                  [36, 37, 38, 39, 40]]])

In [15]:
print(offsets.shape)
print(bert_outputs.shape)

torch.Size([2, 3, 5])
torch.Size([2, 4, 5])


In [16]:
bert_outputs.gather(1,offsets)

tensor([[[ 1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10],
         [11, 12, 13, 14, 15]],

        [[26, 27, 28, 29, 30],
         [31, 32, 33, 34, 35],
         [36, 37, 38, 39, 40]]])

In [17]:
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())

def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b

def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)
        
def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

In [18]:
class GAPBot(BaseBot):
    def __init__(self, model, train_loader, val_loader, optimizer, clip_grad=0,
        avg_window=100, log_dir="./cache/logs/", log_level=logging.INFO,
        checkpoint_dir="./cache/model_cache/", batch_idx=0, echo=False,
        device="cuda:0", use_tensorboard=False):
        super().__init__(
            model, train_loader, val_loader, 
            optimizer=optimizer, clip_grad=clip_grad,
            log_dir=log_dir, checkpoint_dir=checkpoint_dir, 
            batch_idx=batch_idx, echo=echo,
            device=device, use_tensorboard=use_tensorboard
        )
        self.criterion = torch.nn.CrossEntropyLoss()
        self.loss_format = "%.6f"
        
    def extract_prediction(self, tensor):
        return tensor
    
    # Logs
    def snapshot(self):
        loss = self.eval(self.val_loader)
        loss_str = self.loss_format % loss
        self.logger.info("Snapshot loss %s", loss_str)
        self.logger.tb_scalars(
            "losses", {"val": loss},  self.step)
        target_path = (
            self.checkpoint_dir / "best.pth")        
        if not self.best_performers or (self.best_performers[0][0] > loss):
            torch.save(self.model.state_dict(), target_path)
            self.best_performers = [(loss, target_path, self.step)]
            self.logger.info("Saving checkpoint %s...", target_path)
        else:
            new_loss_str = self.loss_format % self.best_performers[0][0]
            self.logger.info("This performance:%s is not as a good as our previously saved:%s", loss_str,new_loss_str )
        assert Path(target_path).exists()
        return loss

In [20]:
df_train = pd.read_csv("gap-test.tsv", delimiter="\t")
df_val = pd.read_csv("gap-validation.tsv", delimiter="\t")
df_test = pd.read_csv("../content/test_stage_2.tsv", delimiter="\t")
sample_sub = pd.read_csv("../content/sample_submission_stage_2.csv")
assert sample_sub.shape[0] == df_test.shape[0]

In [21]:
print(len(df_train))
df_train.head()

2000


,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [22]:
BERT_MODEL = 'bert-large-uncased'

tokenizer = BertTokenizer.from_pretrained(
    BERT_MODEL,
    do_lower_case=True,
    never_split = ("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]", "[A]", "[B]", "[P]")
)
# These tokens are not actually used, so we can assign arbitrary values.
tokenizer.vocab["[A]"] = -1
tokenizer.vocab["[B]"] = -1
tokenizer.vocab["[P]"] = -1

100%|██████████| 231508/231508 [00:00<00:00, 701213.30B/s]


## Making train,test and validation datasets

In [23]:
train_ds = GAPDataset(df_train, tokenizer)
val_ds = GAPDataset(df_val, tokenizer)
test_ds = GAPDataset(df_test, tokenizer, labeled=False)
# dataset convertation to dataloader
train_loader = DataLoader(
    train_ds,
    collate_fn = collate_examples,     #batch
    batch_size=20,
    num_workers=2,
    pin_memory=True,   # Using lock page memory，so that tensor transfer to cuda will be faster
    shuffle=True,
    drop_last=True     # Drop incomplete batch
)
val_loader = DataLoader(
    val_ds,
    collate_fn = collate_examples,
    batch_size=128,
    num_workers=2,
    pin_memory=True,
    shuffle=False
)
test_loader = DataLoader(
    test_ds,
    collate_fn = collate_examples,
    batch_size=128,
    num_workers=2,
    pin_memory=True,
    shuffle=False
)

In [24]:
len(train_loader), len(test_loader), len(val_loader)

(100, 97, 4)

In [25]:
next(iter(test_loader))

[tensor([[  101,  2005,  1996,  ...,     0,     0,     0],
         [  101,  2044,  2023,  ...,     0,     0,     0],
         [  101,  1999,  1996,  ...,     0,     0,     0],
         ...,
         [  101,  1999,  1996,  ...,     0,     0,     0],
         [  101,  1999, 15331,  ...,     0,     0,     0],
         [  101,  2044,  1996,  ...,     0,     0,     0]]),
 tensor([[ 18,  39,  69],
         [ 49,  65,  89],
         [ 48,  81,  72],
         [ 56,  75,  82],
         [ 55,  77,  81],
         [  6,  25,  32],
         [ 52,  54,  58],
         [ 70,  82,  85],
         [ 57,  76,  80],
         [ 55,  61,  68],
         [ 44,  63,  70],
         [ 51,  61,  67],
         [ 46,  52,  68],
         [ 41,  50,  54],
         [ 46,  54,  66],
         [ 19,  24,  33],
         [ 40,  47,  72],
         [ 43,  58,  64],
         [ 51,  57,  74],
         [ 42,  48,  54],
         [ 51,  62,  68],
         [ 44,  50,  57],
         [ 45,  47,  38],
         [ 36,  49,  63],
      

In [26]:
model = GAPModel(BERT_MODEL, device)
# You can unfreeze the last layer of bert by calling set_trainable(model.bert.encoder.layer[23], True)
set_trainable(model.bert, False)
set_trainable(model.head, True)

100%|██████████| 1248501532/1248501532 [00:39<00:00, 31532978.93B/s]


Initing batchnorm
Initing linear
Initing batchnorm
Initing linear
Initing batchnorm
Initing linear
Initing batchnorm
Initing linear


## Adding hyperparametrs for optimization process

In [27]:
lr=1e-3
weight_decay=5e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

bot = GAPBot(
    model, train_loader, val_loader,
    optimizer=optimizer, echo=True,
    avg_window=25
)

[[06/14/2021 02:39:29 PM]] SEED: 420
[[06/14/2021 02:39:29 PM]] # of paramters: 340,403,203
[[06/14/2021 02:39:29 PM]] # of trainable paramters: 5,261,315


# Train

In [28]:
steps_per_epoch = len(train_loader) 
n_steps = steps_per_epoch * 27
bot.train(
    n_steps,
    log_interval=steps_per_epoch // 4,
    snapshot_interval=steps_per_epoch,
    scheduler=TriangularLR(
        optimizer, max_mul=20, ratio=2, steps_per_cycle=n_steps)
)   

[[06/14/2021 02:39:41 PM]] Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.001
    lr: 5e-05
    weight_decay: 0.005
)
[[06/14/2021 02:39:41 PM]] Batches per epoch: 100
[[06/14/2021 02:39:41 PM]] ====================Epoch 1====================
[[06/14/2021 02:40:34 PM]] Step 25: train 2.052181 lr: 7.533e-05
[[06/14/2021 02:41:28 PM]] Step 50: train 1.928328 lr: 1.017e-04
[[06/14/2021 02:42:21 PM]] Step 75: train 1.922204 lr: 1.281e-04
[[06/14/2021 02:43:13 PM]] Step 100: train 1.870402 lr: 1.545e-04
100%|██████████| 4/4 [00:45<00:00, 11.42s/it]
[[06/14/2021 02:43:59 PM]] Snapshot loss 0.921929
[[06/14/2021 02:44:04 PM]] Saving checkpoint cache/model_cache/best.pth...
[[06/14/2021 02:44:04 PM]] New low

[[06/14/2021 02:44:04 PM]] ====================Epoch 2====================
[[06/14/2021 02:44:58 PM]] Step 125: train 1.822521 lr: 1.809e-04
[[06/14/2021 02:45:51 PM]] Step 150: train 1.786613 lr: 2.073e-04
[[06/14/2021 02:46

In [29]:
torch.save(model.state_dict(), './model.pth')

In [30]:
# Predict function
def predict(loader, *, return_y=False):
    model.eval()
    outputs, y_global = [], []
    with torch.set_grad_enabled(False):
        for input_tensors in loader:
            input_tensors = [x.to(model.device) for x in input_tensors if x is not None]
            outputs.append(bot.predict_batch(input_tensors).cpu())
        outputs = torch.cat(outputs, dim=0)
    return outputs

# Predict

In [31]:
preds = predict(test_loader)
len(preds)

12359

In [32]:
df_sub = pd.DataFrame(torch.softmax(preds, -1).cpu().numpy().clip(1e-3, 1-1e-3), columns=["A", "B", "NEITHER"])
df_sub["ID"] = df_test.ID
df_sub.head()

,A,B,NEITHER,ID
0,0.710140,0.281646,0.008214,000075809a8e6b062f5fb3c191a8ed52
1,0.990411,0.002958,0.006631,0005d0f3b0a6c9ffbd31a48453029911
2,0.060085,0.864929,0.074986,0007775c40bedd4147a0573d66dc28f8
3,0.472420,0.524444,0.003135,001194e3fe1234d00198ef6bba4cc588
4,0.066665,0.927180,0.006154,0014bb7085278ef3f9b74f14771caca9
